In [ ]:
# import necessary packages for the project
import copy,csv,time,nltk,emoji,gensim,string 
import pandas as pd 
import numpy as np 
from tabulate import tabulate  
from nltk import word_tokenize,sent_tokenize
from nltk.corpus import stopwords  
import matplotlib.pyplot as plt 
from gensim.models import word2vec 
import gensim.models.keyedvectors as word2vec #need to use due to depreceated model
from keras.models import Sequential
from keras.preprocessing import sequence
from keras import models,layers  
from keras.layers import Dense,Dropout,LSTM,Activation,Bidirectional,Flatten,Embedding
from keras.optimizers import SGD
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer,one_hot,text_to_word_sequence
from keras.initializers import Constant 
from sklearn import model_selection
from sklearn import metrics 
from sklearn.metrics import classification_report,precision_score,recall_score,f1_score,roc_auc_score,cohen_kappa_score,accuracy_score,confusion_matrix
from sklearn.model_selection import train_test_split,KFold,cross_val_score 
from keras.utils.np_utils import to_categorical
from sklearn.datasets import make_circles
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow import set_random_seed
from sklearn.naive_bayes import MultinomialNB,GaussianNB,BernoulliNB
from sklearn.feature_extraction.text import TfidfVectorizer,TfidfTransformer
from nltk import bigrams, trigrams
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn import preprocessing
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report,confusion_matrix
import warnings
import sys
from collections import defaultdict
from sklearn.feature_extraction.text import CountVectorizer
warnings.filterwarnings("ignore")  #Ignoring unnecessory warnings
# import the dataset to our project 
df=pd.read_csv('commentP_consN.csv',delimiter=';',names=['comment'],encoding="utf8")
all=pd.DataFrame(df) #changing to data frame 
print(all) #display the first n rows of comments
print(all.info()) 

In [ ]:
#describition before preprocessing 
reviews_len = [len(x) for x in df['comment']]
pd.Series(reviews_len).hist()
print(plt.show())
print(pd.Series(reviews_len).describe())
# remove the comment to identify sum of words and maximum word before preprocessing 
'''
df['tokenized_sents']= df.apply(lambda row: nltk.word_tokenize(row['comment']), axis=1)
df['sents_length'] = df.apply(lambda row: len(row['tokenized_sents']),axis=1)
print(df[['tokenized_sents','sents_length']])
print("sum of words is",sum(df['sents_length']))
print("the maximum sents and minimum length is",df['sents_length'].max(),df['sents_length'].min())
'''

In [ ]:
#remove digits 
df['comment'] = df['comment'].str.replace('\d+', '')
#print(df[['comment']])
#the following is normalization of words and characters
df['comment'] = df['comment'].replace(regex={"ዐ":"አ","ዑ":"ኡ","ዒ":"ኢ","ዓ":"አ","ዔ":"ኤ","ዕ":"እ","ዖ":"ኦ",
                                            "ዓ":"አ","ኽ":"ህ","ሃ":"ሀ","ዉ":"ው",
                                            "ጸ":"ፀ","ጹ":"ፁ","ጺ":"ፂ","ጻ":"ፃ","ጼ":"ፄ","ጽ":"ፅ","ጾ":"ፆ",
                                             "ኃ":"ሀ","ኁ":"ሁ","ኂ":"ሂ","ኃ":"ሀ","ኄ":"ሄ","ኅ":"ህ","ኆ":"ሆ",
                                             "ሠ":"ሰ","ሡ":"ሱ","ሢ":"ሲ","ሣ":"ሳ","ሤ":"ሴ","ሥ":"ስ","ሦ":"ሶ",
                                            "ሐ":"ሀ","ሑ":"ሁ","ሒ":"ሂ","ሓ":"ሀ","ሔ":"ሄ","ሕ":"ህ","ሖ":"ሆ",                                             
                                             "ኮመንት":"አስተያየት","ሙቭ":"ፊልም","ኢትዮጽያ":"ኢትዮጵያ","ኢትዬ":"ኢትዮጵያ",
                                             "ኢቲዮፒያ":"ኢትዮጵያ","ፍልም":"ፊልም","በጠም":"በጣም","1ኛ":"አንደኛ","ይመችክ":"ይመችህ",
                                             "ይሁንላቹ":"ይሁንላችሁ","አረ":"እረ","ላይክ":"ዉደድ","ፍቅርርር":"ፍቅር","ፍቅርር":"ፍቅር",
                                             "ሀሪፍ":"አሪፍ","ሀሪፈ":"አሪፍ","ሰውደወ":"ስወደው","ዋውውውው":"ዋው","ኡፍፍፍፍፍፍፍ":"ኡፍፍ","ኡፍፍፍፍፍፍ":"ኡፍፍ",
                                             "ኡፍፍፍፍፍ":"ኡፍፍ","ኡፍፍፍ":"ኡፍ","ውድድድድድ":"ውድድ",
                                            "ውድድድድ":"ውድድ","ውድድድ":"ውድድ","1ኛ":"አንደኛ","ኢትዩቢያዊ":"ኢትዮጽያዊ"})
print(df[['comment']])
#removing htmls from each comment 
df['comment']=df['comment'].replace('{html}',"") 
freq = pd.Series(' '.join(df['comment']).split()).value_counts()[-10:]
# uncomment the next line of code when Emojis are not considered
df['comment'] = [''.join(c for c in s if c not in string.punctuation) for s in df['comment']]
# uncomment the following line of codes when considering Emojis for comment labeling 
''' 
#df['comment'] = df['comment'].str.replace('[^\w\s]','') # for removing Emojis from the dataset uncomment it.
for index, row in df.iterrows():
     row=df['comment']==""
     index_no=df.index[row] #display index numbers(rows) that have element(values) 
aa=df['comment'].drop(index_no,axis=0)
aa.to_csv('output.csv', index=False, header=None)
df=pd.read_csv('output.csv',delimiter=';',names=['comment'],encoding="utf-8")
afteremojiR=pd.DataFrame(df) #changing to data frame
print(afteremojiR)
'''

In [ ]:
#tokenizing of the comments 
df['tokenized_sents']= df.apply(lambda row: nltk.word_tokenize(row['comment']), axis=1)
# removing na values(containing empty values)
df['tokenized_sents'].replace('', np.nan, inplace=True)
#print(tabulate(dft['tokenized_sents']))
frequency=df['tokenized_sents']
# Removing Amharic Stopwords
stop = stopwords.words('Amharic')
df['stopwords'] = df['tokenized_sents'].apply(lambda x: len([item for item in x if item in stop]))
df['tokenized_sents']=df['tokenized_sents'].apply(lambda x: [item for item in x if item not in stop])

In [ ]:
df['sents_length'] = df.apply(lambda row: len(row['tokenized_sents']),axis=1)
print(df[['tokenized_sents','sents_length']])
print("sum of words is",sum(df['sents_length']))
print("the maximum sents length is",df['sents_length'].sum())
df4=df['tokenized_sents']
df5=pd.Series.to_frame(df4) #to change series to frame'
df5.columns=['tokenized_sents']
#dfp=pd.read_csv('positive_emoji.csv',delimiter=';',names=['positive_emoji'],encoding="utf8")
#dfn=pd.read_csv('negative_emoji.csv',delimiter=';',names=['negative_emoji'],encoding="utf8")
df2=pd.read_csv('Negative.csv',names=['negative'],encoding="utf8")
df7=pd.read_csv('positive.csv',names=['positive'],encoding="utf8")
df8=pd.read_csv('Invertor.csv',names=['Invertor'],encoding="utf8")
df9=pd.read_csv('intensifierA.csv',names=['intensifier'],encoding="utf8")
columen_names=['negative','positive','Invertor','intensifierA']
result=pd.concat([df2,df7,df8,df9],axis=1,join='outer',sort=False,names=columen_names)
blankIndex=[''] * len(result)
result.index=blankIndex
#information about lexicons 
print("number of positive lexicon",df8.info())
print("number of negative lexicon",df9.info())

In [ ]:
positive_count=0
negative_count=0
invertor_count=0
intensifier_count=0
pos_emoji=0
positive_emoji_count=0
negative_emoji_count=0
emoji_score=0
#review length analysis used to include or exclude very short or lengthy statements
reviews_len = [len(x) for x in df['comment']]
pd.Series(reviews_len).hist()
print(plt.show())
print(pd.Series(reviews_len).describe())
df10 = pd.DataFrame(columns=['polarity'])
# The following are list of labeled positive and negative Emojis
def extract_emoji(str):
            return ''.join(c for c in str if c in emoji.UNICODE_EMOJI)
positive_emoji=['😂','❤','♥','😍','😘','😊','👌','💕','👏','😁','☺','♡','👍','🙏','✌','😏',
                '😉','🙌','🙈','√', '❤️','♥️','♥️','❤️',
'💪','😄','😃','😱','🎉','😜','🌸','💜','💙','😳','💗','☀','😎','😢','💋','😋','🙊','🎶','💞','💝'
'😌','💯','💛','💁','💚','😆','😝','😅','👊','😀','😚','😻','💘','👋','✋','🎊','❄','😥','😈',
'🔝','😰','⚽','👑','😹','🍃','🎁','🐧','🎈','✊','💤','💓','💦','🙋','🎄','🎵','😛','😬','👯',
'💎','🎂','👫','🏆','☝','😙','⛄','👅','♪','🍂','💏','🌴','👈','🌹','🙆','💰','🍻''🌞','🍁','⭐',
'🎀','🙉','💅','🌺','🐶','🌚','🎤','👭','🎧','👆','🍸','😇','🍉','🎸','🍹','💫','📚','🌷','💝',
'💨','🏈','💍','☔','👸','🇪','😸','🍔','👼','🐯','😵','👶','💐','↾','📖','🐒','🌍','┊','🐥','💄',
'💸','⛔','🏀','💉','💟','😯','♦','🌙','🐟','👣','🗿','🍝','🍭','❌','🐰','💊','🚨','🍪','✧','🎆',
'🎎','🇩','✅','🔊','👠','🌌','🍎','🐻','💇','🍊','🍒','🐭','👟','🌎','🍍','🐮','📲','🌅','🇷','🌽',
'🍬','😺','🚀','¦🍧','🍜','🆗','🍋','➤','🏄','👧','🐏','✏','🌾','🏡','👙','⛅','🍅','📺','🐍','🇮',
'♣🇹','🐬','🌳','💿','🔐','🐨','🌕','🔵','🍳','🚴','👰','⚓','👗','➕','💬','🔜','🍨','🍙','🍗','🍲',
'😼','🐙','👨','🍚','🍖','♨▃','🚘']
negative_emoji=['😭',' 😩','😒','💖','😔','😡','😴','🔫','😞','😪',' 😫','💀','😕',
'💔','😤','😑','😠','😓','😣','😲','😿','😐' '😨','😷','👎','💩','🙅','😶','🔪','💃',
                '👿','✂','👪','😦','🍣','🙍','💧','😾','🍥','😭'] 

In [ ]:
# Rules for labeling comments using labeled lexicons and Emoji
for (index_label, row_series) in df5.iterrows():
    list=row_series.values  #accessing the row values
    for i in range(len(list)):
         #print(list[i])
        ll=len(list[i])
        #print(ll)
        li=list[i]
        print(li)              #if df2['negative'].isin(li[j]) 
        xx=extract_emoji(li)
        #print(xx)    
        y=[]
        for s in xx:
            y.append(s)
            ''.join(y)
        #print(y)
        pos_emojiexist=[]
        neg_emojiexist=[]
        for element in positive_emoji:
            for word in y:
                pos_emojiexist.append(word in element)
                positive_emoji_count=sum(pos_emojiexist)
        for item in negative_emoji:
            for word in y:
                neg_emojiexist.append(word in item)
                negative_emoji_count=sum(neg_emojiexist)
        if len(y)!=0:
            emoji_score=positive_emoji_count-negative_emoji_count
        else:
            emoji_score=0
        positive_res=result.positive.isin(li)
        negative_res=result.negative.isin(li)
        invertor_res=result.Invertor.isin(li)
        intensifierA_res=result.intensifier.isin(li)
        positive_count=sum(positive_res)*2
        negative_count=sum(negative_res)*2
        invertor_count=sum(invertor_res)
        intensifier_count=sum(intensifierA_res)
        sentiment_score=positive_count-negative_count
        if invertor_count>0 and intensifier_count==0 and sentiment_score>0 and  ll>0:
            sentiment=-(sentiment_score)
        if invertor_count>0 and intensifier_count==0 and sentiment_score<0  and ll>0:
            sentiment=-(sentiment_score)
        if intensifier_count>0 and invertor_count==0 and sentiment_score>0  and ll>0:
            sentiment=sentiment_score+3
        if intensifier_count> 0 and invertor_count==0 and sentiment_score <0 and ll>0:
            sentiment=sentiment_score-3
        if invertor_count==0 and intensifier_count==0 and sentiment_score >0 and ll>0:
            sentiment=sentiment_score
        if invertor_count==0 and intensifier_count==0 and sentiment_score <0 and   ll>0:
            sentiment=sentiment_score
        if intensifier_count> 0 and invertor_count> 0 and sentiment_score>0  and ll>0:
            sentiment=-(sentiment_score)+3
        if intensifier_count> 0 and invertor_count> 0 and sentiment_score<0 and ll>0:
            sentiment=-(sentiment_score)-3
        if intensifier_count> 0 and invertor_count==0 and sentiment_score==0 and ll>0:
             sentiment=sentiment_score
        if sentiment_score==0 and invertor_count==0 and intensifier_count==0 and ll>0:
             sentiment=sentiment_score
        if sentiment>0 and emoji_score>=0:
            print("positive")
            df10 = df10.append({'polarity': "positive"}, ignore_index=True)
        if sentiment==0 and emoji_score>0:
            print("positive")
            df10 = df10.append({'polarity': "positive"}, ignore_index=True)
        if sentiment<0 and emoji_score<=0:
            print("negative")
            df10 = df10.append({'polarity': "negative"}, ignore_index=True)
        if sentiment==0 and emoji_score<0:
            print("negative")
            df10 = df10.append({'polarity': "negative"}, ignore_index=True)
        if sentiment<0 and emoji_score>0:#no change word first
            print("negative")
            df10 = df10.append({'polarity': "negative"}, ignore_index=True)
        if sentiment > 0 and emoji_score<0: #word first
            print("positive")
            df10 = df10.append({'polarity': "positive"}, ignore_index=True)
        if sentiment==0 and emoji_score==0:
            print("neutral")
            df10 = df10.append({'polarity': "neutral"}, ignore_index=True)
        if ll==0:
            print("neither")            
df11=pd.concat([df4,df10],axis=1,join='inner',sort=False)  
#print(df11)  

In [ ]:
 #change to numerical values
df11.to_csv('annotated_csv', sep='\t', encoding='utf-8') 
df11['polarity'] = df11.polarity.map(lambda x: int(2) if x =='positive' else int(0) if x =='negative' else int(1) if x == 'neutral' else np.nan)
print(df11['polarity'].value_counts())
print(df11)
plt.hist(df11.polarity, bins = 3, align= 'mid')
plt.xticks(range(3), ['Negative','Neutral', 'Positive'])
plt.xlabel('Sentiment of Reviews')
plt.title('Distribution of Sentiment')
plt.show()

In [ ]:
# For Random upsampling of the manority class(Two make two samples equal)
df12=df11.sample(n=9138, replace=True, random_state=1)
print(df12.shape)

In [ ]:
x=df12['tokenized_sents']
y=df12['polarity']
seed =7
np.random.seed(seed)
maximumlen=420 #maximum tweet length(no of words)
# Truncate and pad the review sequences
#Convert words to integers
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df12['tokenized_sents'].values)
x = tokenizer.texts_to_sequences(df12['tokenized_sents'].values)
vocab_size = len(tokenizer.word_index) + 1 
print("vocab_size",vocab_size)
X = pad_sequences(x,maxlen=maximumlen)
print(X.shape[0])
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.10,random_state=seed)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=seed)#validation data
x_train=pad_sequences(x_train, maxlen=maximumlen)
x_test=pad_sequences(x_test, maxlen=maximumlen)
#padding x_val to maximum length
x_val=pad_sequences(x_val, maxlen=maximumlen)
print(x_train)
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)
model2 = gensim.models.Word2Vec(df11['tokenized_sents'],min_count = 1, size = 32,window = 5, sg = 1,iter=20,max_vocab_size=24200,seed=7) 
print(model2)
#print(model2.vector_size)
model2.save('w2v.model')
#use the following lines of code to show similariy of words using word2vec
#print(model2.similarity('ጥሩ','ምርጥ'))
#print(model2.similarity('💚','💋'))
#print(model2.wv.most_similar('💋'))

In [ ]:
topwords=15000 #top words to consider
epoch=8         
batch=14
seed=7
embedding_size = 8
model=Sequential()
np.random.seed(0)
model.add(Embedding(topwords,8,input_length=maximumlen,dropout=0.5))
model.add(LSTM(8,return_sequences=True,input_shape=(x_train.shape[0],x_train.shape[1])))
model.add(Dropout(0.7))
#model.add(LSTM(4,activation='softmax',return_sequences=True))
model.add(Flatten())
model.add(Dense(16, activation='sigmoid'))
model.add(Dropout(0.5))
model.add(Dense(3, activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
history=model.fit(x_train, y_train,validation_data=(x_val,y_val),epochs=epoch,batch_size=batch,verbose=2,
                  callbacks=[EarlyStopping(monitor='val_loss', patience=5, baseline=0.3)])
print(model.summary()) 
loss, accuracy = model.evaluate(x_test, y_test,verbose = 2, batch_size=batch) 
print("Training Accuracy: {:.4f}".format(accuracy))
plt.style.use('ggplot')
def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('epoch')
    plt.ylabel('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('epoch number')
    plt.ylabel('Training and validation loss')
    plt.legend()
plot_history(history)

In [ ]:
# Displaying the Evaluation Metrics values 
yhat_probs = model.predict(x_test, verbose=0)
yhat_probs = np.array([np.argmax(pred) for pred in yhat_probs])
# predict crisp classes for test set
yhat_classes = model.predict_classes(x_test, verbose=0)
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(y_test, yhat_classes,normalize=True)
print('Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(y_test, yhat_classes,labels=[0,1,2],average='macro') #average='micro' gives total score of Tp/tp+fp none gives each class precision
print('Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(y_test, yhat_classes,labels=[0,1,2],average='macro')
print('Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(y_test, yhat_classes,labels=[0,1,2],average='macro')
print('F1 score: %f' % f1)
# confusion matrix
matrix = confusion_matrix(y_test,yhat_classes,labels=[0,1,2])
print(matrix)

In [ ]:
# Displaying the confusion matrics
import itertools
def plot_confusion_matrix(cm, classes, normalize=False, title='Confusion Matrix', cmap=plt.cm.Blues):
    plt.figure()
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    print(cm)
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j], horizontalalignment="center", color="white" if cm[i, j] > thresh else "black")
    plt.tight_layout()
    plt.ylabel('True')
    plt.xlabel('Predicted')
# plot the confusion Matrix
cm = confusion_matrix(y_test,yhat_classes)
plot_confusion_matrix(cm, {'negative': 0, 'neutral':1,'positive':2})